# 🚀 VerifDoc Engine - Colab Environment (Synthetic Edition)

Ce notebook inclut l'Usine à Faux (Synthetic Factory) pour transformer 10 documents en 10 000.

In [ ]:
# 1. Installation des Dépendances (OCR & PDF Tools)
!apt-get install -y poppler-utils tesseract-ocr libtesseract-dev libgl1-mesa-glx
!pip install fastapi uvicorn python-multipart pikepdf Pillow pytesseract numpy opencv-python-headless fpdf requests google-generativeai

In [ ]:
# 2. Importation des Fichiers
from google.colab import files
import os

print("👇 Cliquez sur le bouton 'Choisir les fichiers' ci-dessous.")
print("⚠️ Uploadez 'verifdoc_colab_pack_lite.zip' (Version Légère).")

uploaded = files.upload()

for fn in uploaded.keys():
  print(f'✅ Reçu : {fn} ({len(uploaded[fn])} octets)')

In [ ]:
# 3. Décompression (FIXED: Pas de double dossier)
import os

pack_name = "verifdoc_colab_pack.zip"
if os.path.exists("verifdoc_colab_pack_lite.zip"):
    pack_name = "verifdoc_colab_pack_lite.zip"

if os.path.exists(pack_name):
    print(f"📦 Installation de {pack_name}...")
    !unzip -o {pack_name}
    print("✅ Moteur installé.")
else:
    print("❌ ERREUR : verifdoc_colab_pack_lite.zip manquant ! Uploadez-le.")


In [ ]:
# 4. 🏭 USINE À FAUX (Auto-Path Fix)
# Cette cellule génère 1000 faux pour chaque vrai doc trouvé.

import os
import random
from PIL import Image, ImageDraw, ImageFilter

# --- AUTO-CORRECTION DES CHEMINS ---
base_path = 'verifdoc_engine'
if os.path.exists('verifdoc_engine/verifdoc_engine/DATASET_REAL'):
    base_path = 'verifdoc_engine/verifdoc_engine'
    print("⚠️ Structure double détectée, correction appliquée.")

REAL_DIR = os.path.join(base_path, 'DATASET_REAL/AUTHENTIC')
FAKE_DIR = os.path.join(base_path, 'DATASET_REAL/TAMPERED')
MASK_DIR = os.path.join(base_path, 'DATASET_REAL/MASKS')
# -----------------------------------

if not os.path.exists(FAKE_DIR): os.makedirs(FAKE_DIR)
if not os.path.exists(MASK_DIR): os.makedirs(MASK_DIR)

def create_forgery(image_path, output_path, mask_path, severity=1):
    try:
        with Image.open(image_path) as img:
            img = img.convert("RGB")
            w, h = img.size
            
            # Copier-Coller aléatoire
            rw, rh = random.randint(40, 150), random.randint(20, 60)
            x1, y1 = random.randint(0, w - rw), random.randint(0, h - rh)
            x2, y2 = random.randint(0, w - rw), random.randint(0, h - rh)
            
            region = img.crop((x1, y1, x1+rw, y1+rh))
            img_fake = img.copy()
            img_fake.paste(region, (x2, y2))
            
            # Masque
            mask = Image.new("L", (w, h), 0)
            draw = ImageDraw.Draw(mask)
            draw.rectangle([x2, y2, x2+rw, y2+rh], fill=255)
            
            img_fake.save(output_path)
            mask.save(mask_path)
            return True
    except Exception as e:
        # print(f"Skip {image_path}: {e}")
        return False

# Execution
if not os.path.exists(REAL_DIR):
     print(f"❌ ERREUR CRITIQUE : Dossier introuvable : {REAL_DIR}")
     print("Essayez de RELANCER la Cellule 3 pour voir.")
else:
    files = [f for f in os.listdir(REAL_DIR) if f.lower().endswith('.jpg')]
    print(f"🎯 Trouvé {len(files)} Vrais Docs dans {REAL_DIR}. Lancement du Multiplicateur x100...")

    total_generated = 0
    for f in files:
        for i in range(100): # x100 par image = 1600 images
            in_p = os.path.join(REAL_DIR, f)
            out_p = os.path.join(FAKE_DIR, f"{f}_fake_{i}.jpg")
            mask_p = os.path.join(MASK_DIR, f"{f}_mask_{i}.png")
            if create_forgery(in_p, out_p, mask_p):
                total_generated += 1
            
    print(f"✅ TERMINE. {total_generated} Faux générés. Dataset total > {total_generated} images.")

In [ ]:
# 5. Démarrage du Serveur & Bench
import subprocess
import time
import os

# Auto-path pour le serveur
server_dir = 'verifdoc_engine'
if os.path.exists('verifdoc_engine/verifdoc_engine/main.py'):
    server_dir = 'verifdoc_engine/verifdoc_engine'

os.chdir(server_dir)
print(f"🚀 Démarrage du moteur depuis {server_dir}...")
proc = subprocess.Popen(['uvicorn', 'main:app', '--host', '0.0.0.0', '--port', '8000'])
time.sleep(5)
print("✅ Serveur en ligne.")